In [4]:
import pandas as pd
import numpy as np
import seaborn as sns

In [5]:
vgsales = pd.read_csv("vgsales.csv")

In [8]:
n = pd.DataFrame(vgsales.isna().sum())
m = n.loc[n[0]!=0]
m['missing'] = m[0]*100/len(vgsales.index)
m

C:\Users\srdes\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,0,missing
Year,271,1.632727
Publisher,58,0.349440


In [12]:
vgsales_c = vgsales.dropna()

In [13]:
n = pd.DataFrame(vgsales_c.isna().sum())
m = n.loc[n[0]!=0]
m['missing'] = m[0]*100/len(vgsales.index)
m

,0,missing


In [14]:
vgsales_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16291 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16291 non-null  int64  
 1   Name          16291 non-null  object 
 2   Platform      16291 non-null  object 
 3   Year          16291 non-null  float64
 4   Genre         16291 non-null  object 
 5   Publisher     16291 non-null  object 
 6   NA_Sales      16291 non-null  float64
 7   EU_Sales      16291 non-null  float64
 8   JP_Sales      16291 non-null  float64
 9   Other_Sales   16291 non-null  float64
 10  Global_Sales  16291 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.5+ MB


In [19]:
cate_feat = [col for col in vgsales_c.columns if vgsales_c[col].dtypes == 'O']
cate_feat

['Name', 'Platform', 'Genre', 'Publisher']

In [23]:
cate_unique = list(map(lambda x: vgsales_c[x].nunique(), cate_feat))
l = list(zip(cate_feat, cate_unique))
l

[('Name', 11325), ('Platform', 31), ('Genre', 12), ('Publisher', 576)]

By no surprise the Name category has a lot of unique values

In [25]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

In [27]:
df1 = vgsales_c.copy()
cate_feat[1]

'Platform'

In [28]:
encoder = LabelEncoder()
#df1['Platform'] = encoder.fit_transform(df1[cate_feat[1]]) 
df1['Platform'] = encoder.fit_transform(df1[cate_feat[1]])
#df1['Genre'] = encoder.fit_transform(df1[cate_feat[2]])
df1['Genre'] = encoder.fit_transform(df1[cate_feat[2]])

In [34]:
df1['Publisher'] = df1['Publisher'].replace('<','', inplace=True)
df1['Publisher'] = encoder.fit_transform(df1[cate_feat[3]])

In [35]:
X = df1.drop(['Global_Sales', 'Name'], axis=1)
y = df1['Global_Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

tree = DecisionTreeRegressor(random_state=1)
tree.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=1, splitter='best')

In [36]:
pred = tree.predict(X_test)
mae = mean_absolute_error(y_test, pred)
print("\033[32mMean Absolute Error: {}\033[00m" .format(mae)) 

Mean Absolute Error: 0.018509698011294186
